In [1]:
import http.client
import json
from settings import API_KEY
import time
from pandas.io.json import json_normalize
import pandas as pd
import  csv
import ast

## Funções

In [11]:
def get_url_credits(movie_id, api_key):
    return("/3/movie/" + str(movie_id) + "/credits" + "?api_key=" + str(api_key))

def get_json(conn, url):
	payload = "{}"
	conn.request("GET", url, payload)
	res = conn.getresponse()
	data = res.read()
	decoded = json.loads(data.decode("    utf-8"))    
	header = res.getheader('Retry-After')
	return(decoded, header)

In [63]:
def get_movie_credits(ids):
    
	conn = http.client.HTTPSConnection("api.themoviedb.org")    
	movie_credits_df = pd.DataFrame()
    
	for movie_id in ids:
		url = get_url_credits(movie_id, API_KEY)
		movie_data, header = get_json(conn, url)
		print("Movie ID: ", movie_id, " Header: ", header)
        
		if ('status_code' in movie_data):
			time.sleep(int(header)+1)  
			movie_data, header = get_json(conn, url)        
			print("Movie ID: ", movie_id, "Header: ", header)            
        
		df = json_normalize(movie_data)
		movie_credits_df = movie_credits_df.append(df)

	return movie_credits_df

In [16]:
def get_cast(ids, movie_credits_df):

    movie_cast_df = pd.DataFrame()

    print(movie_credits_df['id'])
    for idx,movie_id in enumerate(movie_credits_df['id']):
    #for movie_id in ids_num:

        movie_item = pd.DataFrame()
        #print(movie_credits_df.head(90))
        print("movie_id: %s" % (movie_id))
        movie_item = movie_credits_df[movie_credits_df['id']==movie_id].reset_index()
        #print(movie_item.head(90))
        print("%s of %s" % (idx,len(ids)))

        #print(movie_item['cast'])
        
        for cast_id in movie_item['cast'][0]:
            cast_id.update({'movie_id': int(movie_id)})
            cast_id.pop('profile_path', None)
            movie_cast_df = movie_cast_df.append(cast_id, ignore_index=True)
    
    return movie_cast_df

## Testes

In [7]:
with open('out_sample.csv', 'r') as f:
#with open('out_all.csv', 'r') as f:    
    reader = csv.reader(f, skipinitialspace=True, delimiter=',')
    ids_list = list(reader)
    
ids = [item for sublist in ids_list for item in sublist]

In [8]:
len(ids)

156

In [9]:
# Transformando ids em numérico
ids_num = list(map(int, ids))

In [64]:
start_time = time.time()
movie_credits_df = get_movie_credits(ids_num)
print("--- Movie Credits: %s ---" % (time.time() - start_time))
print("")

Movie ID:  77  Header:  None
Movie ID:  98  Header:  None
Movie ID:  1359  Header:  None
Movie ID:  8358  Header:  None
Movie ID:  7443  Header:  None
Movie ID:  955  Header:  None
Movie ID:  107  Header:  None
Movie ID:  2133  Header:  None
Movie ID:  6282  Header:  None
Movie ID:  40096  Header:  None
Movie ID:  146  Header:  None
Movie ID:  11688  Header:  None
Movie ID:  2024  Header:  None
Movie ID:  10862  Header:  None
Movie ID:  10393  Header:  None
Movie ID:  9532  Header:  None
Movie ID:  71  Header:  None
Movie ID:  1907  Header:  None
Movie ID:  16187  Header:  None
Movie ID:  4247  Header:  None
Movie ID:  4244  Header:  None
Movie ID:  9679  Header:  None
Movie ID:  10473  Header:  None
Movie ID:  6521  Header:  None
Movie ID:  4327  Header:  None
Movie ID:  19457  Header:  None
Movie ID:  2440  Header:  None
Movie ID:  8859  Header:  None
Movie ID:  10562  Header:  None
Movie ID:  2123  Header:  None
Movie ID:  3176  Header:  None
Movie ID:  1493  Header:  None
Movie ID:

In [65]:
movie_credits_df.shape

(156, 3)

In [66]:
movie_credits_df['id'].isnull().sum()

0

In [68]:
movie_credits_df['id'].duplicated().sum()

0